# 11. 항공 사진 내 선인장 식별
## 11.3 베이스라인 모델
* 딥러닝 모델링 절차
    * 시드값 고정 및 GPU 장비 설정
        * 시드값 고정 : 결과 재현을 위한 작업
        * GPU 장비 설정 : 훈련 속도를 높이기 위해 데이터를 GPU가 처리하도록 변경
    * 데이터 준비
        * 훈련/검증 데이터 분리
        * 데이터셋 클래스 정의 : 이미지 데이터를 모델링에 적합한 형태로 불러오도록 해줌
        * 데이터셋 생성
        * 데이터 로더(데이터셋으로부터 데이터를 배치 단위로 불러와주는 객체) 생성
    * 모델 생성
        * 신경망 모델 클래스를 설계한 후 인스턴스 생성
    * 모델 훈련
        * 손실 함수와 옵티마이저 설정
        * 모델 훈련 : 신경망의 가중치를 갱신하며 모델 훈련
    * 성능 검증
    * 예측 및 제출
### 11.3.1 시드값 고정 및 GPU 장비 설정

In [28]:
import torch
import random
import numpy as np
import os

In [29]:
# 시드값 고정
seed = 50
os.environ['PYTHONHASHSEED'] = str(seed)
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed) # cpu
torch.cuda.manual_seed(seed) # gpu
torch.cuda.manual_seed_all(seed) # multi gpu
torch.backends.cudnn.deterministic = True # 확정적 연산 사용
torch.backends.cudnn.benchmark = False # 벤치마크 기능 해제
torch.backends.cudnn.enabled = False # cudnn 사용 해제

In [30]:
# 장비 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

### 11.3.2 데이터 준비

In [31]:
# 데이터 준비
import pandas as pd

data_path = '../../data/11_cactus/'
labels = pd.read_csv(data_path + 'train.csv')
submission = pd.read_csv(data_path + 'sample_submission.csv')
labels.head()

,id,has_cactus
0,0004be2cfeaba1c0361d39e2b000257b.jpg,1
1,000c8a36845c0208e833c79c1bffedd1.jpg,1
2,000d1e9a533f62e55c289303b072733d.jpg,1
3,0011485b40695e9138e92d0b3fb55128.jpg,1
4,0014d7a11e90b62848904c1418fc8cf2.jpg,1


In [32]:
from zipfile import ZipFile

with ZipFile(data_path+'train.zip') as zipper:
    zipper.extractall(data_path)
with ZipFile(data_path+'test.zip') as zipper:
    zipper.extractall(data_path)

In [33]:
# 훈련/검증 데이터 분리
from sklearn.model_selection import train_test_split

train, valid = train_test_split(
    labels,
    test_size=0.1,
    stratify=labels['has_cactus'],
    random_state=50
)

In [34]:
# 데이터셋 클래스 정의
import cv2
from torch.utils.data import Dataset

def identity(X):
    return X

class ImageDataset(Dataset):
    def __init__(self, df, img_dir='./', transform=None):
        super().__init__()
        self.df = df
        self.img_dir = img_dir
        self.transform = transform if transform else identity

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_id = self.df.iloc[idx, 0]
        img_path = self.img_dir + img_id
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        label = self.df.iloc[idx, 1]
        image = self.transform(image)
        return image, label

In [35]:
# 데이터셋 생성
from torchvision import transforms

transform = transforms.ToTensor()
dataset_train = ImageDataset(df=train, img_dir=data_path+'train/', transform=transform)
dataset_valid = ImageDataset(df=valid, img_dir=data_path+'train/', transform=transform)
dataset_train[0][0].shape

torch.Size([3, 32, 32])

In [36]:
# 데이터 로더 생성
from torch.utils.data import DataLoader

loader_train = DataLoader(dataset=dataset_train, batch_size=32, shuffle=True)
loader_valid = DataLoader(dataset=dataset_valid, batch_size=32, shuffle=False)

### 11.3.3 모델 생성

In [37]:
import torch.nn as nn
import torch.nn.functional as F

In [38]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=2)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=2)
        self.max_pool = nn.MaxPool2d(kernel_size=2)
        self.avg_pool = nn.AvgPool2d(kernel_size=2)
        self.fc = nn.Linear(in_features=64*4*4, out_features=2)

    def forward(self, x):
        x = self.max_pool(F.relu(self.conv1(x))) # (n,3,32,32) => (n,32,17,17), 17 <=pooling= 34 = (32+2*2-3) / 1 + 1
        x = self.max_pool(F.relu(self.conv2(x))) # (n,32,17,17) => (n,64,9,9), 9 <=pooling= 19 = (17+2*2-3) / 1 + 1
        x = self.avg_pool(x) # (n,64,9,9) => (n,64,4,4)
        x = x.view(-1, 64 * 4 * 4)
        x = self.fc(x)
        return x

model = Model().to(device)
model

Model(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
  (max_pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (avg_pool): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (fc): Linear(in_features=1024, out_features=2, bias=True)
)

### 11.3.4 모델 훈련

In [39]:
# 손실 함수
criterion = nn.CrossEntropyLoss()

In [40]:
# 옵티마이저
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

In [42]:
epochs = 10

for epoch in range(epochs):
    epoch_loss = 0
    for images, labels in loader_train:
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad() # 기울기 초기화
        outputs = model(images)
        loss = criterion(outputs, labels)
        epoch_loss += loss.item()
        loss.backward()
        optimizer.step() # 새로운 가중치 = 기존 가중치 - (학습률 * 기울기)
    print(f'에폭 [{epoch+1}/{epochs}] - 손실값 : {epoch_loss/len(loader_train):.4f}')

에폭 [1/10] - 손실값 : 0.5239
에폭 [2/10] - 손실값 : 0.3400
에폭 [3/10] - 손실값 : 0.2410
에폭 [4/10] - 손실값 : 0.1965
에폭 [5/10] - 손실값 : 0.1811
에폭 [6/10] - 손실값 : 0.1653
에폭 [7/10] - 손실값 : 0.1522
에폭 [8/10] - 손실값 : 0.1403
에폭 [9/10] - 손실값 : 0.1360
에폭 [10/10] - 손실값 : 0.1298


### 11.3.5 성능 검증

In [43]:
from sklearn.metrics import roc_auc_score
true_list = []
preds_list = []

In [44]:
model.eval()
with torch.no_grad():
    for images, labels in loader_valid:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        preds = torch.softmax(outputs.cpu(), dim=1)[:, 1]
        true = labels.cpu()
        preds_list.extend(preds)
        true_list.extend(true)

print(f'Valid ROC AUC : {roc_auc_score(true_list, preds_list):.4f}')

Valid ROC AUC : 0.9902


### 11.3.6 예측 및 결과 제출

In [45]:
# test dataset & loader
dataset_test = ImageDataset(df=submission, img_dir=data_path+'test/', transform=transform)
loader_test = DataLoader(dataset_test, batch_size=32, shuffle=False)

In [48]:
# predict
preds = []
model.eval()
with torch.no_grad():
    for images, _ in loader_test:
        images = images.to(device)
        outputs = model(images)
        preds_part = torch.softmax(outputs.cpu(), dim=1)[:, 1].tolist() # tensor -> list
        preds.extend(preds_part)

In [51]:
# 결과 제출
submission['has_cactus'] = preds
submission.to_csv(data_path+'submission.csv', index=False)

In [50]:
# 파일 삭제
import shutil
shutil.rmtree(data_path+'train')
shutil.rmtree(data_path+'test')